# Graph Machine Learning para identificação dinâmica de comunidades em grafos

## Aplicação às comunidades de Produção Acadêmica

In [62]:
from neo4j import GraphDatabase
from py2neo import Graph

class Neo4jConnection:

    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

class Neo4jMetricsExtractor:

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def distinct_community_names(self):
        with self._driver.session() as session:
            result = session.run("""
            MATCH (n:GrandeÁrea)
            RETURN DISTINCT n.name AS community_name
            """)
            return [record["community_name"] for record in result]
    
    def distinct_community_codes(self):
        with self._driver.session() as session:
            result = session.run("""
            MATCH (n:GrandeÁrea)
            RETURN DISTINCT n.code AS community_code
            """)
            return [record["community_code"] for record in result]

    def community_density(self, community_code):
        with self._driver.session() as session:
            result = session.run("""
            MATCH (n:GrandeÁrea) WHERE n.code = $community_code
            WITH COUNT(n) AS node_count
            MATCH (n:GrandeÁrea)-[r]-() WHERE n.code = $community_code
            WITH node_count, COUNT(r) AS rel_count
            RETURN CASE WHEN node_count = 0 THEN 0 ELSE rel_count * 1.0 / node_count END AS density
            """, community_code=community_code)
            
            record = result.single()
            if record:
                return record["density"]
            else:
                raise ValueError(f"No data found for community: {community_code}")
                
    def average_semantic_cohesion(self, community_code):
        with self._driver.session() as session:
            result = session.run("""
            MATCH (ga:GrandeÁrea {code: $community_code})-[:CONTÉM_ÁREA]->(:Área)-[:CONTÉM_SUBÁREA]->(:Subárea)-[:CONTÉM_ESPECIALIDADE]->(h)
            WHERE (h:Especialidade OR h:Subárea)
            WITH collect(h) AS hierarchies
            UNWIND hierarchies AS hierarchy
            MATCH (p:Publicacao)-[r:SIMILAR]->(hierarchy)
            WITH COUNT(r) AS total_relations, SIZE(hierarchies) AS total_hierarchies
            RETURN total_relations * 1.0 / total_hierarchies AS average_semantic_cohesion
            """, community_code=community_code)
            record = result.single()
            return record["average_semantic_cohesion"] if record else None

    def predominant_hierarchy(self, community_code):
        with self._driver.session() as session:
            result = session.run("""
            MATCH (n) WHERE n.code = $community_code
            RETURN labels(n) AS node_labels, COUNT(*) AS frequency
            ORDER BY frequency DESC
            LIMIT 1
            """, community_code=community_code)
            record = result.single()
            predominant_label = [label for label in record["node_labels"] if label in ["GrandeÁrea", "Área", "Subárea", "Especialidade"][0]]
            return predominant_label, record["frequency"]


    def community_name_density(self, community_name):
        with self._driver.session() as session:
            result = session.run("""
            MATCH (n:GrandeÁrea) WHERE n.name = $community_name
            WITH COUNT(n) AS node_count
            MATCH (n:GrandeÁrea)-[r]-() WHERE n.name = $community_name
            WITH node_count, COUNT(r) AS rel_count
            RETURN CASE WHEN node_count = 0 THEN 0 ELSE rel_count * 1.0 / node_count END AS density
            """, community_name=community_name)
            
            record = result.single()
            if record:
                return record["density"]
            else:
                raise ValueError(f"No data found for community: {community_name}")

    def average_name_semantic_cohesion(self, community_name):
        with self._driver.session() as session:
            result = session.run("""
            MATCH (n1:GrandeÁrea)-[r]->(n2:GrandeÁrea) WHERE n1.name = $community_name AND n2.name = $community_name
            RETURN AVG(r.similarity) AS average_semantic_cohesion
            """, community_name=community_name)
            return result.single()["average_semantic_cohesion"]

    def predominant_name_hierarchy(self, community_name):
        with self._driver.session() as session:
            result = session.run("""
            MATCH (n:GrandeÁrea) WHERE n.name = $community_name
            RETURN n.hierarchy, COUNT(*) AS frequency
            ORDER BY frequency DESC
            LIMIT 1
            """, community_name=community_name)
            record = result.single()
            return record["n.hierarchy"], record["frequency"]   

    def count_all_nodes(self):
        with self._driver.session() as session:
            result = session.run("""
            MATCH (n)
            RETURN count(n) as qte_nodes
            """)
            record = result.single()
            return record["qte_nodes"] if record else 0
    
    def count_unconnected_publications(self):
        with self._driver.session() as session:
            result = session.run("""
            MATCH (p:Publicacao)
            WHERE NOT (p)--()
            RETURN COUNT(p) AS unconnected_publications_count
            """)
            record = result.single()
            return record["unconnected_publications_count"] if record else 0
    
    def create_named_projection(self):
        with self._driver.session() as session:
            # Suponho que estamos considerando todas as relações e nodos e que a propriedade de score é relevante
            query = """
            CALL gds.graph.create('similarity_graph', '*', 
            {
                ALL: {
                    type: '*', 
                    properties: 'score'
                }
            })
            """
            session.run(query)

    def delete_named_projection(self):
        with self._driver.session() as session:
            session.run("CALL gds.graph.drop('similarity_graph')")

    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response
    
    def calculate_louvain(self, score_threshold):
        with self._driver.session() as session:
            # First, create the named graph projection
            session.run("""
            CALL gds.graph.project.cypher(
              'louvainGraph',
              'MATCH (n) RETURN id(n) AS id',
              'MATCH (n1)-[r:SIMILAR]->(n2) WHERE r.score > $scoreThreshold RETURN id(n1) AS source, id(n2) AS target, r.score AS weight',
              {parameters: {scoreThreshold: $scoreThreshold}}
            )
            YIELD graphName
            """, scoreThreshold=score_threshold)
            
            # Then, compute Louvain communities on the projection
            result = session.run("""
            CALL gds.louvain.stream('louvainGraph')
            YIELD nodeId, communityId
            RETURN gds.util.asNode(nodeId).name AS nodeName, communityId
            """)
            
            # Drop the graph projection after the analysis
            session.run("CALL gds.graph.drop('louvainGraph')")
            
            # Return the Louvain computation result
            return [(record["nodeName"], record["communityId"]) for record in result]
        
    def calculate_louvain_and_modularity(self, score_threshold):
        with self._driver.session() as session:
            # Crie a projeção do grafo com base no limiar especificado
            session.run("""
            CALL gds.graph.create.cypher(
                'dynamic_similarity_graph',
                'MATCH (n) RETURN id(n) AS id',
                'MATCH (n1)-[r:SIMILAR]->(n2) WHERE r.score >= $score_threshold RETURN id(n1) AS source, id(n2) AS target, r.score AS weight',
                { parameters: { score_threshold: $score_threshold } }
            )
            """, score_threshold=score_threshold)

            # Execute o algoritmo Louvain na projeção do grafo criada
            louvain_result = session.run("""
            CALL gds.louvain.stream('dynamic_similarity_graph', { includeIntermediateCommunities: false })
            YIELD nodeId, communityId
            RETURN gds.util.asNode(nodeId).name AS name, communityId
            """)
            
            # Colete os resultados de Louvain
            communities = list(louvain_result)

            # Calcule a modularidade
            modularity_result = session.run("""
            CALL gds.louvain.mutate('dynamic_similarity_graph', {
                mutateProperty: 'louvain',
                includeIntermediateCommunities: false
            })
            YIELD communityCount, modularities
            RETURN communityCount, modularities[0] AS modularity
            """)
            modularity = modularity_result.single()

            # Delete a projeção do grafo após o uso
            session.run("CALL gds.graph.drop('dynamic_similarity_graph')")

            return communities, modularity        

In [38]:
# Criar uma instância do extrator
extractor = Neo4jMetricsExtractor("bolt://localhost:7687", "neo4j", "password")

# Contagem de nós
qte_nodes = extractor.count_all_nodes()
print(qte_nodes)

2394


In [70]:
# Identificar comunidades com Algoritmo de Louvain

extractor = Neo4jMetricsExtractor("bolt://localhost:7687", "neo4j", "password")
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
communities_by_threshold = extractor.calculate_louvain(thresholds)
for i in communities_by_threshold[:5]:
    print(i)
extractor.close()

('Ciências Exatas e da Terra', 0)
('Matemática', 1)
('Elderly', 2)
('Set', 3)
('Mathematics Logic', 4)


In [69]:
extractor = Neo4jMetricsExtractor("bolt://localhost:7687", "neo4j", "password")

isolated_production_nodes = extractor.count_unconnected_publications()
print(f"Quantidade total de nós Produção isolados: {isolated_production_nodes}")

# Get distinct community codes
community_codes = extractor.distinct_community_codes()

# For each community code, calculate and print the metrics
for code in community_codes:
    try:
        density = extractor.community_density(code)
        cohesion = extractor.average_semantic_cohesion(code)
        hierarchy, frequency = extractor.predominant_hierarchy(code)
        print(f"\nCode {code}:")
        print(f"- Density: {density}")
        print(f"- Cohesion: {cohesion}")
        print(f"- Predominant Hierarchy: {hierarchy} (Frequency: {frequency})")
    except ValueError as e:
        print(f"Error for community {code}: {e}")


Quantidade total de nós Produção isolados: 26

Code 1.00.00.00:
- Density: 8.0
- Cohesion: 24.494897959183675
- Predominant Hierarchy: ['GrandeÁrea'] (Frequency: 1)

Code 2.00.00.00:
- Density: 13.0
- Cohesion: 6.617647058823529
- Predominant Hierarchy: ['GrandeÁrea'] (Frequency: 1)

Code 3.00.00.00:
- Density: 13.0
- Cohesion: 11.45531914893617
- Predominant Hierarchy: ['GrandeÁrea'] (Frequency: 1)

Code 4.00.00.00:
- Density: 9.0
- Cohesion: 8.3125
- Predominant Hierarchy: ['GrandeÁrea'] (Frequency: 1)

Code 5.00.00.00:
- Density: 7.0
- Cohesion: 18.608695652173914
- Predominant Hierarchy: ['GrandeÁrea'] (Frequency: 1)

Code 6.00.00.00:
- Density: 13.0
- Cohesion: 28.8062015503876
- Predominant Hierarchy: ['GrandeÁrea'] (Frequency: 1)

Code 7.00.00.00:
- Density: 10.0
- Cohesion: 31.072164948453608
- Predominant Hierarchy: ['GrandeÁrea'] (Frequency: 1)

Code 8.00.00.00:
- Density: 3.0
- Cohesion: 0.8260869565217391
- Predominant Hierarchy: ['GrandeÁrea'] (Frequency: 1)


In [ ]:
extractor = Neo4jMetricsExtractor("bolt://localhost:7687", "neo4j", "password")

# Get distinct community codes
community_names = extractor.distinct_community_names()

# For each community code, calculate and print the metrics
for name in community_names:
    try:
        density = extractor.community_name_density(name)
        cohesion = extractor.average_name_semantic_cohesion(name)
        hierarchy, frequency = extractor.predominant_name_hierarchy(name)
        print(f"Community {name}:")
        print(f"- Density: {density}")
        print(f"- Cohesion: {cohesion}")
        print(f"- Predominant Hierarchy: {hierarchy} (Frequency: {frequency})")
    except ValueError as e:
        print(f"Error for community {name}: {e}")


In [ ]:
import os
import time
import psutil
import torch
import optuna
import logging
import numpy as np
import networkx as nx
from tqdm.notebook import tqdm
from community import community_louvain
from torch.nn import functional as F
from torch_geometric.nn import GraphSAGE
from torch_geometric.utils import to_networkx
from torch_geometric.data import DataLoader, Data
from py2neo import Graph, ServiceUnavailable, Neo4jError

class CosineSimilarityRelationship:
    def __init__(self, uri, user, password, model_name="default_model"):
        self.uri = uri
        self.user = user
        self.password = password
        self.connect()

    def connect(self):
        try:
            self.graph = Graph(self.uri, auth=(self.user, self.password))
        except ServiceUnavailable as e:
            logging.error(f"Failed to connect to the database. Reason: {e}")
            raise ServiceUnavailable(f"Failed to connect to the database. Reason: {e}")

    def check_connection(self):
        if not hasattr(self, 'graph') or not self.graph:
            logging.error("No database connection")
            raise Exception("No database connection")

    def cosine_similarity(self, a, b):
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

    def normalize_similarity(self, similarity):
        return (similarity + 1) / 2

    def get_memory_usage(self):
        process = psutil.Process(os.getpid())
        memory_MB = process.memory_info().rss / (1024 ** 2)
        return memory_MB

    def get_all_embeddings(self, label):
        cursor = self.graph.run(f"MATCH (n:{label}) RETURN id(n) AS id, n.embedding AS embedding")
        return [record for record in cursor]

    def process_similarity_for_nodes(self, source_embeddings, target_embeddings, source_label, target_label, threshold, batch_size):
        relationships_created_count = 0
        node_pairs_count = 0
        tx = self.graph.begin()
        try:
            for source in tqdm(source_embeddings, desc=f"Processing Semantic Similarity {source_label}/{target_label}"):
                for target in target_embeddings:
                    similarity = self.cosine_similarity(np.array(source["embedding"]), np.array(target["embedding"]))
                    normalized_similarity = self.normalize_similarity(similarity)
                    node_pairs_count += 1
                    if similarity > threshold:
                        query = f"""
                        MATCH (source:{source_label}) WHERE id(source) = $source_id
                        MATCH (target:{target_label}) WHERE id(target) = $target_id
                        MERGE (source)-[:SIMILAR {{score: $similarity, weight: $normalized_similarity}}]->(target)
                        """
                        tx.run(query, source_id=source["id"], target_id=target["id"], similarity=float(similarity), normalized_similarity=float(normalized_similarity))
                        relationships_created_count += 1
                        if relationships_created_count % batch_size == 0:
                            tx.commit()
                            logging.info(f"Committed {batch_size} relationships for {source_label}/{target_label}.")
                            tx = self.graph.begin()
            if tx:
                tx.commit()
                logging.info(f"Committed remaining relationships for {source_label}/{target_label}.")
        except Exception as e:
            logging.error(f"An error occurred during transaction: {e}")
            tx.rollback()
            if tx:
                tx.rollback()
        return node_pairs_count, relationships_created_count

    def create_similarity_embeedings_relationships(self, threshold=0.7, batch_size=3000):
        start_time = time.time()
        self.check_connection()
        initial_memory = self.get_memory_usage()

        pub_embeddings = self.get_all_embeddings("Publicacao")
        esp_embeddings = self.get_all_embeddings("Especialidade")
        sub_embeddings = self.get_all_embeddings("Subárea")

        total_pub = len(pub_embeddings)
        total_esp = len(esp_embeddings)
        total_sub = len(sub_embeddings)

        logging.info(f"Nodes: Publicacao {total_pub}, Especialidade {total_esp}, Subárea {total_sub}")

        pub_sub_pairs, pub_sub_rels = self.process_similarity_for_nodes(pub_embeddings, sub_embeddings, "Publicacao", "Subárea", threshold, batch_size)
        pub_esp_pairs, pub_esp_rels = self.process_similarity_for_nodes(pub_embeddings, esp_embeddings, "Publicacao", "Especialidade", threshold, batch_size)

        final_memory = self.get_memory_usage()
        end_time = time.time()
        memory_difference = final_memory - initial_memory
        processing_time = end_time - start_time

        logging.info(f"RAM Consumption: {np.round(memory_difference,2)} MB")
        logging.info(f"Processing Time: {np.round(processing_time,2)} seconds")
        logging.info(f"Current Memory Usage: {np.round(final_memory,2)} MB")
        logging.info(f"Execution time for similarity calculations and relationship creation: {np.round(processing_time, 2)} seconds")
        logging.info(f"Similarity threshold: {threshold}")
        logging.info(f"Total node pairs analyzed: {pub_sub_pairs + pub_esp_pairs}")
        logging.info(f"Node pairs Publicacao/Subárea: {pub_sub_pairs}")
        logging.info(f"Node pairs Publicacao/Especialidade: {pub_esp_pairs}")
        logging.info(f"Total relationships created: {pub_sub_rels + pub_esp_rels}")

    def run_similarity_operations(self, threshold=0.7):
        self.create_similarity_embeedings_relationships(threshold)

class DatabaseConnectionManager:
    def __init__(self, uri, user, password):
        self.uri = uri
        self.user = user
        self.password = password

    def get_graph(self):
        try:
            return Graph(self.uri, auth=(self.user, self.password))
        except ServiceUnavailable as e:
            logging.error(f"Failed to connect to the database. Reason: {e}")
            raise

class GraphDataRetriever:
    def __init__(self, uri, user, password):
        try:
            self.graph = Graph(uri, auth=(user, password))
        except Neo4jError as e:
            print("Error connecting to the Neo4j database:", e)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    def retrieve_graph_data(self, threshold):
        try:
            query = """
            MATCH (p:Publicacao)-[s:SIMILAR]->(t)
            WHERE s.score >= {}
            RETURN p, s, t
            """.format(threshold)
            results = self.graph.run(query)

            node_features, edge_features, edge_index = [], [], []

            for record in results:
                p, s, t = record['p'], record['s'], record['t']
                degree = len(list(self.graph.match((p, None, None))))
                node_features.append([degree])
                edge_features.append([s['score']])
                edge_index.append([p.id, t.id])

            node_features = torch.tensor(node_features, dtype=torch.float).to(self.device)
            edge_features = torch.tensor(edge_features, dtype=torch.float).to(self.device)
            edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous().to(self.device)

            # Compute global graph features
            # Centralidade dos Nós
            degree_centrality_query = """
            MATCH (n:Publicacao)
            RETURN n.name AS node, size((n)--()) AS degree
            """
            degree_centrality = self.graph.run(degree_centrality_query).data()

            # Coeficiente de Aglomeração (Clustering Coefficient)
            clustering_coefficient_query = """
            MATCH (n:Publicacao)
            WITH n, apoc.node.degree(n) AS degree, apoc.coll.toSet([neighbor IN apoc.neighbors.nodes(n) | neighbor]) AS neighbors
            WHERE degree > 1
            RETURN n.name AS node, toFloat(size(apoc.coll.pairsMin(neighbors)[index IN range(0, size(apoc.coll.pairsMin(neighbors))-2) | 
            CASE WHEN (neighbors[index])-->(neighbors[index+1]) THEN 1 ELSE 0 END]) + size(apoc.coll.pairsMin(neighbors)[index IN range(0, 
            size(apoc.coll.pairsMin(neighbors))-2) | CASE WHEN (neighbors[index+1])-->(neighbors[index]) THEN 1 ELSE 0 END]))/toFloat((degree*(degree-1))) AS clustering
            """
            clustering_coefficient = self.graph.run(clustering_coefficient_query).data()

            # Densidade do Grafo
            graph_density_query = """
            MATCH (n:Publicacao), (m:Publicacao) WHERE id(n) < id(m)
            RETURN toFloat(count(distinct(n, m))) / (toFloat(count(n)*count(n)-1)/2) AS graph_density
            """
            graph_density = self.graph.run(graph_density_query).data()[0]['graph_density']

            # Transitividade
            graph_transitivity_query = """
            CALL apoc.metrics.clusteringCoefficient()
            YIELD clusteringCoefficient
            RETURN clusteringCoefficient
            """
            graph_transitivity = self.graph.run(graph_transitivity_query).data()[0]['clusteringCoefficient']

            # Número de Componentes Conexas
            connected_components_query = """
            CALL apoc.algo.unionFind('Publicacao')
            YIELD partition
            RETURN count(distinct partition) AS number_of_components
            """
            num_connected_components = self.graph.run(connected_components_query).data()[0]['number_of_components']

            # Assortatividade
            assortativity_query = """
            CALL apoc.metrics.assortativity.degree('Publicacao', 'INCOMING', 'INCOMING')
            YIELD assortativityCoefficient
            RETURN assortativityCoefficient
            """
            assortativity = self.graph.run(assortativity_query).data()[0]['assortativityCoefficient']

            # Integração com o objeto Data
            data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
            data.degree_centrality = torch.tensor([node['degree'] for node in degree_centrality], dtype=torch.float)
            data.clustering_coefficient = torch.tensor([node['clustering'] for node in clustering_coefficient], dtype=torch.float)
            data.graph_density = torch.tensor([graph_density], dtype=torch.float)
            data.graph_transitivity = torch.tensor([graph_transitivity], dtype=torch.float)
            data.num_connected_components = torch.tensor([num_connected_components], dtype=torch.int)
            data.assortativity = torch.tensor([assortativity], dtype=torch.float)

            return data

        except Neo4jError as e:
            print("Error retrieving graph data:", e)
            return None

    def create_dataset(self, thresholds):
        dataset = []
        for t in thresholds:
            data = self.retrieve_graph_data(t)
            dataset.append(data)
        return dataset

class DataLoaderExtension(CosineSimilarityRelationship):
    def __init__(self, uri, user, password, model_name="default_model"):
        super().__init__(uri, user, password, model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def _create_tensors_from_nodes_and_edges(self, nodes, edges):
        """Transforms nodes and edges into PyTorch tensors."""
        try:
            x = torch.tensor([node['embedding'] for node in nodes], dtype=torch.float).to(self.device)
            edge_index = torch.tensor([edge[:2] for edge in edges], dtype=torch.long).t().contiguous().to(self.device)
            edge_weights = torch.tensor([edge[2] for edge in edges], dtype=torch.float).to(self.device)
            return x, edge_index, edge_weights
        except Exception as e:
            logging.error(f"Error in _create_tensors_from_nodes_and_edges: {e}")
            raise

    def graph_to_pytorch_data(self):
        """Fetches graph data from Neo4j and returns it as PyTorch geometric data."""
        try:
            pub_cursor = self.graph.run("MATCH (p:Publicacao) RETURN id(p) AS id, p.embedding AS embedding")
            all_nodes = list(pub_cursor)

            if not all_nodes:
                logging.warning("No nodes were found in the database.")
                return None

            rel_cursor = self.graph.run("""
            MATCH (p1:Publicacao)-[r:SIMILAR]->(p2)
            RETURN id(p1) AS source, id(p2) AS target, r.score AS weight
            """)
            # RETURN id(p1) AS source, id(p2) AS target, COALESCE(r.score, 1) AS weight
            # RETURN id(p1) AS source, id(p2) AS target, r.score AS weight
            all_edges = [(record['source'], record['target'], record['weight']) for record in rel_cursor]

            x, edge_index, edge_weights = self._create_tensors_from_nodes_and_edges(all_nodes, all_edges)
            data = Data(x=x, edge_index=edge_index, edge_attr=edge_weights)
            
            return data

        except Exception as e:
            logging.error(f"Error in graph_to_pytorch_data: {e}")
            return None

    def create_data_loader(self, batch_size=32):
        """Creates a DataLoader with the provided batch size."""
        dataset = [self.graph_to_pytorch_data()]
        return DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True if self.device.type == "cuda" else False)
        
    def calculate_modularity_neo4j(self):
        """Computes modularity using Neo4j's Louvain algorithm implementation."""
        try:
            query = """
            CALL gds.louvain.stream({
                nodeProjection: 'Publicacao',
                relationshipProjection: {
                    SIMILAR: {
                        type: 'SIMILAR',
                        properties: 'weight',
                        orientation: 'UNDIRECTED'
                    }
                },
                includeIntermediateCommunities: false
            })
            YIELD nodeId, communityId
            RETURN gds.modularity(nodeId, communityId) as modularity
            """
            
            result = self.graph.run(query).single()
            modularity = result['modularity']
            
            return modularity
        
        except Exception as e:
            logging.error(f"Error in calculate_modularity_neo4j: {e}")
            return None

class GraphClassificationModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        """Initialization of the Graph Classification Model."""
        super(GraphClassificationModel, self).__init__()
        self.conv1 = GraphSAGE(in_channels, hidden_channels)
        self.conv2 = GraphSAGE(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        """Forward propagation logic."""
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

    def create_graph_with_threshold(self, threshold):
        """
        Retorna os dados do grafo utilizando DataLoaderExtension e aplica o threshold fornecido.
        
        Args:
        - threshold (float): Limiar para determinar a relação entre os nós.
        
        Returns:
        - data (PyTorch Data Object): Dados do grafo formatados para PyTorch.
        """
        # Incorporando a classe CosineSimilarityRelationship
        similarity_relationship = CosineSimilarityRelationship()
        
        # Utilizando a classe DataLoaderExtension para obter os dados do grafo
        loader_extension = DataLoaderExtension(self.uri, self.user, self.password)
        
        # Gerando os dados do grafo com o threshold aplicado
        data = loader_extension.graph_to_pytorch_data(threshold, similarity_relationship)
        
        return data

    def objective(self, trial):
        """Objective function for hyperparameter tuning using Optuna."""
        threshold = trial.suggest_float('threshold', 0.1, 1.0)
        data = self.create_graph_with_threshold(threshold)

        loader = DataLoader([data], batch_size=32, shuffle=True)

        num_features = data.x.size(1)
        num_classes = len(set(data.y.tolist())) # Assuming data.y contains labels.

        model = GraphClassificationModel(num_features, 128, num_classes)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        criterion = torch.nn.CrossEntropyLoss()

        for epoch in range(100):
            for batch in loader:
                optimizer.zero_grad()
                out = model(batch.x, batch.edge_index)
                loss = criterion(out, batch.y)
                loss.backward()
                optimizer.step()

        G = to_networkx(data)
        partition = community_louvain.best_partition(G)
        modularity = community_louvain.modularity(partition, G)

        return modularity

class CommunityDetection:
    """
    This class is responsible for identifying and evaluating communities within graph-structured data.
    It utilizes the DynamicGraphLearning and DataLoaderExtension classes to extract meaningful representations
    of nodes and subsequently uses these representations for community detection.
    """
    
    def __init__(self, uri, user, password, learning_rate=0.01):
        """
        Constructor for the CommunityDetection class.

        Parameters:
        - uri (str): URI for the database connection.
        - user (str): Username for the database connection.
        - password (str): Password for the database connection.
        - learning_rate (float): Learning rate for the graph learning model.
        """
        self.dynamic_graph_learning = GraphClassificationModel(uri, user, password, learning_rate)
        self.data_loader_extension = DataLoaderExtension(uri, user, password)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    def detect_communities(self, output):
        """
        Detects communities within a graph based on the provided node representations.

        Parameters:
        - output (Tensor): Node representations generated by a graph neural network.

        Returns:
        - dict: Mapping of node indices to their respective community.
        """
        data = self.data_loader_extension.graph_to_pytorch_data()
        G = to_networkx(data, to_undirected=True)
        
        # Using the embeddings as node attributes for community detection.
        for i, node in enumerate(G.nodes()):
            G.nodes[node]['embedding'] = output[i].cpu().numpy()
        
        partition = community_louvain.best_partition(G, weight='embedding')
        
        return partition

    def evaluate_communities(self):
        """
        Evaluates the quality of the detected communities by computing their modularity.

        Returns:
        - float: Modularity score of the detected communities.
        """
        data = self.data_loader_extension.graph_to_pytorch_data()
        model = GraphClassificationModel(data.x.size(1), 128, len(set(data.y.tolist())))
        model.to(self.device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=self.dynamic_graph_learning.learning_rate)
        criterion = torch.nn.CrossEntropyLoss()

        for epoch in range(100):  # An optimal number of epochs can be used.
            optimizer.zero_grad()
            out = model(data.x.to(self.device), data.edge_index.to(self.device))
            loss = criterion(out, data.y.to(self.device))
            loss.backward()
            optimizer.step()

        output = model(data.x.to(self.device), data.edge_index.to(self.device))
        communities = self.detect_communities(output)
        
        G = to_networkx(data, to_undirected=True)
        modularity = community_louvain.modularity(communities, G)
        
        return modularity
    
    def run(self):
        """
        Executes the community detection and evaluation pipeline.

        Returns:
        - float: Modularity score of the detected communities.
        """
        self.dynamic_graph_learning.objective(optuna.trial.FixedTrial({}))
        modularity_score = self.evaluate_communities()
        
        return modularity_score

# Cálculo de Louvain

No contexto do Graph Data Science (GDS) Library do Neo4j, a projeção de nó, especificada pela chave nodeProjection em uma consulta Cypher, define quais rótulos de nó ou tipos de nó devem ser considerados ao realizar algoritmos de análise de grafos. Esta projeção determina um subconjunto de nós do banco de dados que será utilizado durante a execução do algoritmo.

A nodeProjection é uma parte crucial na preparação de um grafo em memória, onde os algoritmos do GDS operam. Quando você chama um procedimento do GDS, como o gds.louvain.stream, é frequentemente necessário informar ao algoritmo qual parte do grafo ele deve considerar. Isso é feito através das projeções de nó e de relação.

Aqui está um detalhamento do termo nodeProjection: 'Node':

    nodeProjection: É a chave utilizada na chamada do procedimento GDS para definir quais nós serão incluídos na projeção do grafo.

    'Node': Este é o valor associado à chave e refere-se ao rótulo de nó específico no banco de dados Neo4j que você deseja incluir na projeção. Por exemplo, se o seu banco de dados Neo4j tem nós rotulados como Person, Company, Product, etc., ao especificar nodeProjection: 'Person', você está dizendo ao algoritmo para considerar apenas os nós que têm o rótulo Person.

A projeção de nó pode ser simples, utilizando apenas um rótulo de nó, ou mais complexa, envolvendo múltiplos rótulos e propriedades que definem filtros ou pesos para os nós no grafo projetado. Além disso, a projeção pode ser especificada de forma mais detalhada usando uma estrutura de mapa (ou dicionário em termos de Python) para fornecer informações adicionais, como propriedades de nós específicas a serem incluídas ou excluídas na projeção do grafo.

Assim, ao realizar análises de grafos no Neo4j utilizando a biblioteca GDS, é essencial especificar corretamente a nodeProjection para que o algoritmo opere sobre o conjunto correto de dados.

In [ ]:
# conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="password")

# # Define the query with placeholders for parameters
# query_with_params = """
# CALL gds.graph.project.cypher(
#   'graphName',
#   'MATCH (n) RETURN id(n) AS id',
#   'MATCH (n1)-[r:SIMILAR]->(n2) WHERE r.score > $scoreThreshold RETURN id(n1) AS source, id(n2) AS target, r.score AS weight',
#   {parameters: {scoreThreshold: $scoreThreshold}}
# )
# YIELD graphName
# CALL gds.louvain.stream('graphName')
# YIELD nodeId, communityId
# RETURN gds.util.asNode(nodeId).name AS name, communityId
# """

# # Define the parameters as a dictionary
# params = {
#     'scoreThreshold': 0.7
# }

# # Execute the query with the parameters
# conn.query(query_with_params, parameters=params)


Para aproveitar a mesma projeção e calcular a modularidade para cada conjunto de comunidades geradas por diferentes limiares de similaridade (threshold), uma estratégia eficiente é criar projeções de grafos parametrizadas e utilizar o algoritmo de modularidade disponível no Neo4j Graph Data Science (GDS) Library. 

A modularidade é uma medida que quantifica a força da divisão de uma rede em módulos (também chamados de comunidades). Quanto maior a modularidade, mais precisa é a divisão em comunidades.

Segue um exemplo de como estender a classe Neo4jMetricsExtractor para calcular a modularidade após a detecção de comunidades com o algoritmo Louvain:

# Cálculo da Modularidade junto com identificação de comunidades por Louvain

No método para cálculo Modularidade junto com a identificação de comunidades:

    Cria projeção de grafo com base no limiar de score passado como argumento. A projeção é dinâmica e temporária, ideal para cálculos que dependem de parâmetros variáveis.
    
    Executa o algoritmo Louvain na projeção para detectar comunidades.
    
    Calcula a modularidade com a função gds.louvain.mutate e retornamos as comunidades detectadas e o valor da modularidade.
    
    Remove a projeção do grafo após seu uso para liberar recursos.
    
É importante notar que esta função assume que a biblioteca GDS está instalada e que você está utilizando a API correta conforme sua versão. As versões mais recentes do GDS podem ter métodos diferentes para calcular modularidades e trabalhar com projeções de grafos. Verifique sempre a documentação oficial para as funções mais atualizadas.

# GML com PyTorch Geometric

Incorporar as métricas do grafo no PyTorch Geometric (PyG) requer uma abordagem que converta esses dados em formatos compatíveis para o aprendizado profundo. O PyG, especificamente, espera que os dados estejam em uma forma particular, tipicamente usando a classe Data para representar gráficos.

Aqui está um exemplo simplificado de como você pode fazer isso, tendo em conta as métricas discutidas anteriormente:

Estrutura básica do objeto Data:
A classe Data no PyTorch Geometric é um contêiner para gráficos. Para gráficos simples, ela contém atributos como edge_index (uma matriz 2xN de índices de arestas), x (uma matriz NxM de características dos nós) e y (uma matriz Nx1 de rótulos dos nós, embora isso possa ser omitido em configurações não supervisionadas).

Incorporar as métricas no objeto Data:
Para o aprendizado dinâmico, pode-se supor que os gráficos estejam mudando ao longo do tempo. As métricas do grafo poderiam, portanto, ser usadas para enriquecer as características dos nós em cada gráfico temporal.

Uso no aprendizado dinâmico não supervisionado:
Para aprendizado dinâmico não supervisionado com auto-aprendizagem, o processo geral seria:

    Usar um encoder para obter embeddings dos nós.
    Utilizar esses embeddings para realizar agrupamento (por exemplo, usando k-means).
    Usar os clusters como pseudo-rótulos.
    Treinar um modelo de classificação usando esses pseudo-rótulos.
    Iterar, refinando os pseudo-rótulos e otimizando o modelo.

As métricas do grafo podem ser usadas para enriquecer as características dos nós durante esse processo, potencialmente fornecendo informações contextuais que melhoram o desempenho do auto-aprendizagem.

Embora a inclusão de métricas de gráfico como características possa enriquecer o modelo, é crucial realizar uma análise e validação adequadas para garantir que essas características realmente contribuam para a melhoria do desempenho do modelo.

Um ponto crucial é que o PyTorch Geometric normalmente opera sobre características de nós e arestas. As métricas que estamos discutindo aqui são características de grafo (ou seja, uma única métrica para todo o grafo). Assim, ao usar essas métricas em um modelo PyG, certifique-se de que o modelo é adequado para características de grafo, ou considere formas de incorporar essas métricas em características de nível de nó ou aresta, conforme necessário.

Além disso, ao trabalhar com aprendizado dinâmico, lembre-se de que as características do grafo podem mudar ao longo do tempo, então as métricas e características devem ser recalculadas conforme o grafo é atualizado.

Dada a complexidade e especificidade do seu problema, o preparo do objeto `Data` para ser ingerido no PyTorch Geometric (PyG) envolverá várias etapas. Vamos abordá-las em sequência:

1. **Projeção Dinâmica do Grafo com Valores de Threshold**: 
   Comece por gerar várias projeções do grafo para diferentes valores de threshold. Cada projeção representará um grafo onde as arestas entre os nós `Publicacao` e `Subárea`/`Especialidade` têm um valor de similaridade semântica acima do threshold definido.

2. **Características do Nó e Aresta**:
   Para cada nó `Publicacao`, calcule a similaridade semântica como uma característica de nó. Para cada aresta entre `Publicacao` e `Subárea`/`Especialidade`, use o valor da similaridade como uma característica de aresta.

3. **Separação de Comunidades**:
   Você pode utilizar algoritmos como o Louvain para identificar comunidades no grafo. Calcule o número de comunidades e o número de nós `Publicacao` que não pertencem a nenhuma comunidade como características globais.

4. **Características Globais**:
   Como o objetivo é espelhar as oito `Grande Área`, você pode também incluir o número total de comunidades identificadas como uma característica global.

5. **Construção do Objeto Data**:
   Cada projeção do grafo (para cada valor de threshold) será representada por um objeto `Data` no PyG. Supondo que você tem matrizes de adjacência e características de nó para cada projeção:

   ```python
   from torch_geometric.data import Data
   import torch

   def create_data_object(edge_index, node_features, edge_features, global_features):
       data = Data(x=torch.tensor(node_features, dtype=torch.float),
                   edge_index=torch.tensor(edge_index, dtype=torch.long),
                   edge_attr=torch.tensor(edge_features, dtype=torch.float))
       data.global_attr = torch.tensor(global_features, dtype=torch.float)
       return data
   ```

6. **Dataset Dinâmico**:
   Após construir um objeto `Data` para cada projeção, compile-os em um conjunto de dados (dataset). Em situações dinâmicas, você pode tratar cada projeção como um "snapshot" no tempo e treinar seu modelo para entender como as características do grafo evoluem à medida que o threshold muda.

7. **Modelo**:
   Utilize um modelo de aprendizado profundo adequado para aprendizado não supervisionado em grafos, como Graph Autoencoders ou Graph Neural Networks (GNN). A entrada seria as características do nó e aresta, e a saída pode ser a reconstituição do grafo ou alguma outra representação útil. Durante o treinamento, você pode usar uma função de perda que promova a identificação de oito comunidades e penalize a exclusão de nós `Publicacao`.

Finalmente, este é um esboço inicial e, dada a complexidade do problema, é provável que ajustes e iterações sejam necessários conforme você avança. Lembre-se também de normalizar ou padronizar as características para facilitar o treinamento do modelo.

In [76]:
import json
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
from torch_geometric.data import InMemoryDataset

class Neo4jService:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def fetch_data(self, query):
        with self._driver.session() as session:
            return session.run(query).data()
        
class DataLoaderExtension:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def fetch_nodes_and_edges(self):
        try:
            node_cursor = self.graph.run("MATCH (p:Publicacao) RETURN id(p) AS id, p.embedding AS embedding")
            rel_cursor = self.graph.run("""
            MATCH (p1:Publicacao)-[r:SIMILAR]->(p2)
            RETURN id(p1) AS source, id(p2) AS target, r.score AS weight
            """)

            nodes = list(node_cursor)
            edges = [(record['source'], record['target'], record['weight']) for record in rel_cursor]

            return nodes, edges
        except Exception as e:
            logging.error(f"Error in fetch_nodes_and_edges: {e}")
            return [], []

    def transform_to_tensors(self, nodes, edges):
        try:
            x = torch.tensor([node['embedding'] for node in nodes], dtype=torch.float).to(self.device)
            edge_index = torch.tensor([edge[:2] for edge in edges], dtype=torch.long).t().contiguous().to(self.device)
            edge_weights = torch.tensor([edge[2] for edge in edges], dtype=torch.float).to(self.device)
            return x, edge_index, edge_weights
        except Exception as e:
            logging.error(f"Error in transform_to_tensors: {e}")
            return None, None, None

    def load_graph_data(self):
        nodes, edges = self.fetch_nodes_and_edges()
        if not nodes:
            logging.warning("No nodes were found in the database.")
            return None

        x, edge_index, edge_weights = self.transform_to_tensors(nodes, edges)
        data = Data(x=x, edge_index=edge_index, edge_attr=edge_weights)
        return data

    def create_data_loader(self, batch_size=32):
        dataset = [self.load_graph_data()]
        return DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True if self.device.type == "cuda" else False)

    def create_graph(self, thresholds):
        """
        Função para criar grafos baseados em diferentes limiares de similaridade de cosseno.

        Parâmetros:
        - uri (str): URI do banco de dados Neo4j.
        - user (str): Nome do usuário do banco de dados.
        - password (str): Senha do usuário do banco de dados.
        - model_name (str): Nome do modelo para gerar embeddings. Padrão é "default_model".
        - thresholds (list of float): Lista de limiares para determinar similaridade. Padrão é [0.6, 0.7, 0.8, 0.9].
        - batch_size (int): Tamanho do lote para inserção em lote no banco de dados. Padrão é 3000.

        Retorna:
        - None
        """
        cosine_sim_rel = CosineSimilarityRelationship(self.graph.uri, self.graph.user, self.graph.password)

        all_graph_data = {}

        for threshold in thresholds:
            # Utilizando o método 'run_similarity_operations' para criar relações de similaridade
            cosine_sim_rel.run_similarity_operations(threshold)
            
            # Obtendo nós e arestas após a criação das relações
            nodes, edges = self.fetch_nodes_and_edges()
            
            if not nodes:
                logging.warning(f"No nodes were found in the database for threshold: {threshold}.")
                continue
            
            x, edge_index, edge_weights = self.transform_to_tensors(nodes, edges)
            
            # Armazenando os dados do grafo para o threshold atual
            all_graph_data[threshold] = Data(x=x, edge_index=edge_index, edge_attr=edge_weights)

        return all_graph_data
            
class GraphService:
    def __init__(self, graph_data):
        self.graph_data = graph_data

    def get_data(self):
        # Esta função pega os dados do grafo e os transforma no formato adequado
        edge_index = torch.tensor(self.graph_data['edges'], dtype=torch.long)
        x = torch.tensor(self.graph_data['nodes'], dtype=torch.float)
        y = torch.tensor(self.graph_data['labels'], dtype=torch.long)
        return Data(x=x, edge_index=edge_index.t().contiguous(), y=y)

    def split_data(self, data, train_percent=0.7):
        # Esta função divide os dados em conjuntos de treinamento e teste
        dataset = InMemoryDataset(root='./', transform=self.get_data)
        train_size = int(train_percent * len(dataset))
        test_size = len(dataset) - train_size
        train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
        return DataLoader(train_dataset, batch_size=64, shuffle=True), DataLoader(test_dataset, batch_size=64, shuffle=False)

class GraphEmbeddingModel(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, dropout=0.5):
        super(GraphEmbeddingModel, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.dropout = dropout

    def forward(self, x, edge_index):
        # Primeira camada de convolução
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        # Segunda camada de convolução
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        # Terceira camada de convolução
        x = self.conv3(x, edge_index)
        return x

class GNNModel(torch.nn.Module):
    """
    Definição do modelo Graph Neural Network usando PyTorch Geometric.
    """
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

class GraphModel:
    """
    Classe representando o modelo de aprendizado profundo para grafos com PyTorch Geometric.
    """
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int):
        self.model = GNNModel(input_dim, hidden_dim, output_dim)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01)

    def train(self, train_data, epochs: int = 10):
        self.model.train()
        for epoch in range(epochs):
            for data in train_data:  # Assumindo que train_data é um DataLoader
                self.optimizer.zero_grad()
                out = self.model(data)
                loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
                loss.backward()
                self.optimizer.step()

    def predict(self, data):
        self.model.eval()
        with torch.no_grad():
            logits = self.model(data)
        return logits

    def evaluate(self, data):
        self.model.eval()
        with torch.no_grad():
            logits = self.model(data)
            pred = logits.argmax(dim=1)
            correct = (pred == data.y).sum().item()
            acc = correct / len(data.y)
        return acc

    def self_learning(self, unlabeled_data):
        # Neste método, a ideia é usar o modelo para fazer previsões em dados não rotulados
        # E então usar essas previsões como novas labels "pseudo" para treinar o modelo novamente
        logits = self.predict(unlabeled_data)
        pseudo_labels = logits.argmax(dim=1)
        unlabeled_data.y = pseudo_labels  # Definir as pseudo labels
        self.train(unlabeled_data)  # Treinar novamente usando as pseudo labels

class GraphAnalysisController:
    def __init__(self, neo4j_service, model_path=None):
        self.neo4j_service = neo4j_service
        # self.graph_data = self._load_data_from_source()
        self.graph_service = GraphService(self.graph_data)
        # Instancie o DataLoaderExtension para carregar dados do Neo4j.
        self.data_loader = DataLoaderExtension(self.graph_data)
        
        # Se o modelo for fornecido, carregue-o.
        if model_path:
            self.graph_model = GraphModel.load(model_path)
        else:
            self.graph_model = GraphModel()

    ## Para usar dados salvos para retreinar o modelo
    # def _load_data_from_source(self):
    #     query = "SEU_QUERY_PARA_OBTENÇÃO_DE_DADOS"
    #     return self.neo4j_service.fetch_data(query)
    
    # def _load_data_from_source(self):
    #     # Para este exemplo, vamos supor uma função simplificada de carregamento de dados
    #     with open(self.data_source_path, 'r') as file:
    #         return json.load(file)

    # def train_model(self, epochs=10):
    #     data = self.graph_service.get_data()
    #     train_loader, test_loader = self.graph_service.split_data(data)
    #     loss_function = torch.nn.CrossEntropyLoss()
    #     optimizer = torch.optim.Adam(self.graph_model.parameters(), lr=0.01)
        
    #     for epoch in range(epochs):
    #         for batch in train_loader:
    #             optimizer.zero_grad()
    #             out = self.graph_model(batch.x, batch.edge_index)
    #             loss = loss_function(out, batch.y)
    #             loss.backward()
    #             optimizer.step()
        
    #     return "Training complete."

    ## Para treinamento não-supervisionado
    def train_model(self, train_data, num_epochs=100):
        optimizer = torch.optim.Adam(self.graph_model.parameters(), lr=0.001)
        
        for epoch in range(num_epochs):
            for batch in train_data:
                optimizer.zero_grad()
                out = self.graph_model(batch.x, batch.edge_index)
                loss = F.mse_loss(out, batch.edge_attr)  # Aqui usamos edge_attr (pesos de borda) em vez de y
                loss.backward()
                optimizer.step()

    def evaluate_model(self, test_data):
        self.graph_model.eval()
        correct = 0
        with torch.no_grad():
            for batch in test_data:
                out = self.graph_model(batch.x, batch.edge_index, batch.edge_attr)
                pred = out.argmax(dim=1)
                correct += int((pred == batch.y).sum())
        return correct / len(test_data.dataset)
    
    def save_model(self, path):
        self.graph_model.save(path)

    def execute_pipeline(self, thresholds=[0.6, 0.7, 0.8, 0.9]):
        # Gere grafos para cada limiar.
        all_graph_data = self.data_loader.create_graph(thresholds)

        # Divida os dados em conjuntos de treinamento e teste.
        graph_service = GraphService(all_graph_data[thresholds[0]])  # Usando o primeiro limiar como exemplo.
        train_data, test_data = graph_service.split_data(all_graph_data[thresholds[0]])

        # Treine o modelo.
        self.train_model(train_data)

        # Avalie o modelo.
        accuracy = self.evaluate_model(test_data)
        print(f"Model accuracy: {accuracy:.4f}")

        # Salve o modelo.
        self.save_model("model_path.pt")

import matplotlib.pyplot as plt
from torch_geometric.data import DataLoader
from sklearn.model_selection import KFold
from IPython.display import clear_output

class GraphTrainingPipeline:
    def __init__(self, data_source_path, uri, username, password, input_dim, hidden_dim, output_dim, batch_size=32, n_splits=5):
        self.data_loader_ext = DataLoaderExtension(uri, username, password)
        self.data_loader = self.data_loader_ext.create_data_loader(batch_size=batch_size)
        self.controller = GraphAnalysisController(data_source_path)
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.n_splits = n_splits
        self.losses = []
        self.accuracies = []

    def train_and_evaluate(self, epochs=10):
        kf = KFold(n_splits=self.n_splits)
        for fold, (train_idx, val_idx) in enumerate(kf.split(self.data_loader.dataset)):
            print(f"Fold {fold + 1}/{self.n_splits}")
            train_subset = torch.utils.data.Subset(self.data_loader.dataset, train_idx)
            val_subset = torch.utils.data.Subset(self.data_loader.dataset, val_idx)
            
            train_loader = DataLoader(train_subset, batch_size=len(train_subset))
            val_loader = DataLoader(val_subset, batch_size=len(val_subset))
            
            self.controller.graph_model = GNNModel(self.input_dim, self.hidden_dim, self.output_dim)  # Reinitialize model for each fold
            
            for epoch in range(epochs):
                self.controller.train_model(epochs=1)  # Train for one epoch
                
                # Plot training loss and validation accuracy in real-time
                self.plot_metrics()

                # Evaluate on validation set and store accuracy
                accuracy = self.controller.evaluate_model()
                self.accuracies.append(accuracy)

                clear_output(wait=True)

    def plot_metrics(self):
        fig, ax = plt.subplots(1, 2, figsize=(12, 5))

        # Plotting losses
        ax[0].plot(self.losses, '-o', label='Training Loss')
        ax[0].set_title('Training Loss')
        ax[0].set_xlabel('Epochs')
        ax[0].set_ylabel('Loss')
        ax[0].legend()

        # Plotting accuracies
        ax[1].plot(self.accuracies, '-o', label='Validation Accuracy')
        ax[1].set_title('Validation Accuracy')
        ax[1].set_xlabel('Epochs')
        ax[1].set_ylabel('Accuracy')
        ax[1].legend()

        plt.tight_layout()
        plt.show()

    def save_model(self, path):
        self.controller.save_model(path)

In [ ]:
class GraphTrainingPipeline:
    # def __init__(self, data_source_path, uri, username, password, input_dim, hidden_dim, output_dim, batch_size=32, n_splits=5):
    def __init__(self, neo4j_uri, neo4j_user, neo4j_password, thresholds=[0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]):
        self.data_loader_extension = DataLoaderExtension(neo4j_uri, neo4j_user, neo4j_password)
        self.graph_data = self.data_loader_extension.create_graph(thresholds)
        self.best_threshold = None
        self.best_accuracy = 0.0
    
        self.graph_model.train() # Define o modelo para o modo de treinamento
        for batch in train_data: # Itera sobre os lotes de dados no DataLoader
            optimizer.zero_grad() # Zera os gradientes
            output = self.graph_model(batch) # Realiza a passagem para a frente

            # # Como o problema parece ser de clusterização, você pode usar uma perda adequada, 
            # # por exemplo, uma perda baseada em similaridade ou uma perda baseada em distância.
            # # Aqui, estou assumindo uma perda genérica, que deve ser substituída pela sua perda específica.
            # loss = some_loss_function(output, target) 
            
            # loss.backward() # Realiza a passagem para trás
            # optimizer.step() # Atualiza os pesos

            # # Você pode querer imprimir o valor da perda após cada época para acompanhar o treinamento
            # print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

    def evaluate_model_threshold(self, thresholds, evaluation_metrics):
        """
        Avalia o modelo para diferentes limiares e retorna o limiar que otimiza as métricas de avaliação.
        
        Parâmetros:
        - thresholds (list of float): Lista de limiares para testar.
        - evaluation_metrics (list of functions): Lista de métricas de avaliação a serem consideradas.
        
        Retorna:
        - best_threshold (float): O limiar que otimizou as métricas de avaliação.
        """

        best_threshold = None
        best_score = float('-inf') 

        for threshold in thresholds:
            current_data = self.data_loader.create_graph([threshold])[threshold]
            score = 0
            for metric in evaluation_metrics:
                score += metric(self.graph_model, current_data)

            if score > best_score:
                best_score = score
                best_threshold = threshold

        return best_threshold

    def community_detection(self, best_threshold):
        """
        Detecta comunidades nos nós Producao usando o limiar otimizado.
        
        Parâmetros:
        - best_threshold (float): O limiar otimizado.
        
        Retorna:
        - communities (list of list of int): Lista de comunidades detectadas.
        """
        # Obter os dados do grafo para o limiar otimizado
        data = self.data_loader.create_graph([best_threshold])[best_threshold]

        # Aplicar o modelo para obter as embeddings dos nós
        embeddings = self.graph_model(data)

        # Use um algoritmo de detecção de comunidade para detectar comunidades nas embeddings
        # Por exemplo, você pode usar o algoritmo Louvain, mas qualquer algoritmo de sua escolha funcionaria.
        partition = community.best_partition(embeddings) # usando python-louvain

        communities = []
        for community_id in set(partition.values()):
            community_nodes = [node for node, c_id in partition.items() if c_id == community_id]
            communities.append(community_nodes)

        return communities

    def save_communities_to_neo4j(self, communities):
        """
        Salva as comunidades detectadas de volta no banco de dados Neo4j.
        
        Parâmetros:
        - communities (list of list of int): Lista de comunidades detectadas.
        
        Retorna:
        - status (str): Status da operação.
        """

        # Aqui, você precisaria escrever o código para salvar as comunidades no banco de dados.
        # Isso pode envolver a criação de relações entre nós em uma comunidade ou a definição de propriedades dos nós com seus IDs de comunidade.

        # Placeholder
        return "Communities saved successfully."

    def run_pipeline(self):
        # Passo 1: Carregar dados
        data = self.data_loader.load_graph_data()

        # Passo 2: Treinar modelo
        self.train_model(data)

        # Passo 3: Avaliar modelo e otimizar o limiar
        thresholds = [i/10 for i in range(5, 10)]
        evaluation_metrics = [some_metric_function] # Adicione suas métricas aqui
        best_threshold = self.evaluate_model_threshold(thresholds, evaluation_metrics)

        # Passo 4: Detectar comunidades usando o limiar otimizado
        communities = self.community_detection(best_threshold)

        # Passo 5: Salvar comunidades no banco de dados Neo4j
        status = self.save_communities_to_neo4j(communities)

        return status


In [ ]:
BATCH_SIZE = 32

# Instanciando a classe para carregamento dos dados
data_loader_ext = DataLoaderExtension(uri="bolt://localhost:7687", user="neo4j", password="your_password")
data_loader = data_loader_ext.create_data_loader(batch_size=BATCH_SIZE)

# Parâmetros para o modelo
INPUT_DIM = len(data_loader.dataset[0].x[0])  # Número de características (features) do nodo
HIDDEN_DIM = 64  # Pode ser ajustado conforme necessidade
OUTPUT_DIM = 10  # Número de classes ou categorias. Ajuste conforme seu dataset

# Instantiate the training pipeline
pipeline = GraphTrainingPipeline(
    DATA_SOURCE_PATH, 
    URI, 
    USERNAME, 
    PASSWORD, 
    INPUT_DIM, 
    HIDDEN_DIM, 
    OUTPUT_DIM
)

# Train and evaluate the model
pipeline.train_and_evaluate(epochs=10)

# Save the trained model
pipeline.save_model("trained_model.pt")

In [ ]:
controller = GraphAnalysisController(neo4j_uri="bolt://localhost:7687", neo4j_user="neo4j", neo4j_password="your_password")
status = controller.run_pipeline()
print(status)